In [ ]:
!pip install xmlschema==1.0.18
!pip install pyarrow==3.0.0

In [ ]:
import boto3
import shutil
import xmlschema
from concurrent.futures import ThreadPoolExecutor 

In [ ]:
s3 = boto3.client('s3')
s3.download_file('wos-bucket', 'xml_to_parquet.py', 'xml_to_parquet.py')
s3.download_file('wos-bucket', 'convert_xml_to_parquet.py', 'convert_xml_to_parquet.py')
s3.download_file('wos-bucket', 'WR_2023_20240112181857_CORE_0001.xml', 'WR_2023_20240112181857_CORE_0001.xml')
s3.download_file('wos-bucket', 'WR_2023_20240112181857_CORE_0002.xml', 'WR_2023_20240112181857_CORE_0002.xml')
s3.download_file('wos-bucket', 'WR_2023_20240112181857_CORE_0003.xml', 'WR_2023_20240112181857_CORE_0003.xml')

In [ ]:
schemas = ['item_dii.rawxml.xsd',
 'item_cabi.rawxml.xsd',
 'item_ccc.rawxml.xsd',
 'BIOSIS_common.rawxml.xsd',
 'item_bp.rawxml.xsd',
 'CHANGELOG',
 'item_bci.rawxml.xsd',
 'summary.rawxml.xsd',
 'fullrecord_metadata.rawxml.xsd',
 'item_medline.rawxml.xsd',
 'item_ba.rawxml.xsd',
 'item_wos.rawxml.xsd',
 'common_types.rawxml.xsd',
 'item_zr.rawxml.xsd',
 'clarivate.com.schema.wok5.30.rawxml.xsd',
 'EWUID.rawxml.xsd',
 'item_fsta.rawxml.xsd',
 'item_rsci.rawxml.xsd',
 'item_drci.rawxml.xsd',
 'item_cscd.rawxml.xsd',
 'item_inspec.rawxml.xsd',
 'item_scielo.rawxml.xsd']

for xsd in schemas:
    s3.download_file('wos-bucket', xsd, xsd)

In [ ]:
# Test xml_to_parquet
s3.download_file('wos-bucket', 'PurchaseOrder.xml', 'PurchaseOrder.xml')
s3.download_file('wos-bucket', 'PurchaseOrder.xsd', 'PurchaseOrder.xsd')
!python xml_to_parquet.py -x PurchaseOrder.xsd PurchaseOrder.xml

In [65]:
# Duplicate test Parquet file to demo parallelism
shutil.copyfile('PurchaseOrder.xml.parquet', 'p2.parquet')
shutil.copyfile('PurchaseOrder.xml.parquet', 'p3.parquet')

par_files = ['PurchaseOrder.xml.parquet', 'p2.parquet', 'p3.parquet']

# Define write function
def to_s3(file):
    body = open(file, 'rb')
    s3r.Bucket('wos-bucket').put_object(
        Key = file,
        Body = body)
    
# Write test Parquet files to S3 in parallel
with ThreadPoolExecutor(max_workers = 3) as executor:
    results = executor.map(to_s3, par_files)

In [ ]:
# Module won't run with WoS XML
# I suspect it's a schema issue; the test example uses a single xsd file,
# while clarivate.com.schema.wok5.30.rawxml.xsd references several other xsd files
!python xml_to_parquet.py -x clarivate.com.schema.wok5.30.rawxml.xsd WR_2023_20240112181857_CORE_0001.xml -l out.txt -v

In [ ]:
# This code should convert the XML files to Parquet files in parallel
# once the schema is correctly specified
schema = 'TBD'
!python xml_to_parquet.py -m 3 -x schema *.xml

In [64]:
# This code would then write the Parquet files to S3 in parallel
par_files = ['WR_2023_20240112181857_CORE_0001.xml',
             'WR_2023_20240112181857_CORE_0002.xml',
             'WR_2023_20240112181857_CORE_0003.xml']

def to_s3(file):
    body = open(file, 'rb')
    s3r.Bucket('wos-bucket').put_object(
        Key = file,
        Body = body)
        
with ThreadPoolExecutor(max_workers = 3) as executor:
    results = executor.map(to_s3, par_files)

IndentationError: unexpected indent (<ipython-input-64-3bc0a714a94c>, line 8)

In [63]:
ls

BIOSIS_common.rawxml.xsd                 item_scielo.rawxml.xsd
CHANGELOG                                item_wos.rawxml.xsd
clarivate.com.schema.wok5.30.rawxml.xsd  item_zr.rawxml.xsd
common_types.rawxml.xsd                  jupyterhub_cookie_secret
convert_xml_to_parquet.py                jupyterhub-proxy.pid
EWUID.rawxml.xsd                         jupyterhub.sqlite
fullrecord_metadata.rawxml.xsd           p2.parquet
item_ba.rawxml.xsd                       p3.parquet
item_bci.rawxml.xsd                      PurchaseOrder.xml
item_bp.rawxml.xsd                       PurchaseOrder.xml.parquet
item_cabi.rawxml.xsd                     PurchaseOrder.xsd
item_ccc.rawxml.xsd                      __pycache__/
item_cscd.rawxml.xsd                     schema.xsd
item_dii.rawxml.xsd                      summary.rawxml.xsd
item_drci.rawxml.xsd                     WR_2023_20240112181857_CORE_0001.xml
item_fsta.rawxml.xsd                     WR_2023_20240112181857_CORE_0002.xml
item_inspec.rawxm